In [2]:
import pandas as pd
import numpy as np
import warnings
from datetime import datetime

warnings.filterwarnings("ignore")

hoy = datetime.now()
año = hoy.year
mes = hoy.month

mes_nombre = hoy.strftime('%b').upper()[:3] # ENE
mes_año = f"{mes_nombre}{str(año)[2:]}" # ENE24

fecha = f"{año}{str(mes).zfill(2)}" # 202401

nombre_base = f"base_asignacion_20241004_ TRABAJADA.xlsx"
asignacion_path = f"bases/asignacion/{fecha}/{nombre_base}"
asignacion_final_path = f"bases/asignacion/{fecha}/base_asignacion_{mes_año}.xlsx"

df_asignacion = pd.read_excel(asignacion_path, sheet_name='BTOTAL')

In [ ]:
df_asignacion_test = df_asignacion.copy()
df_asignacion.shape

In [ ]:
def clean_columns(columns_list: list[str]) -> list[str]:
    return [column.strip().replace('  ', ' ').replace('  ', ' ').replace('.', '').replace(' ', '_').upper() for column in columns_list]

df_asignacion_test.columns = clean_columns(df_asignacion_test.columns)

if 'AGENCIA_CORRECTA' in df_asignacion_test.columns:
    if 'AGENCIA' not in df_asignacion_test.columns:
        df_asignacion_test.rename(columns={'AGENCIA_CORRECTA': 'AGENCIA'}, inplace=True)
    else:
        df_asignacion_test['AGENCIA'] = np.where(
            df_asignacion_test['AGENCIA_CORRECTA'].notna() & (df_asignacion_test['AGENCIA_CORRECTA'].astype(str).str.len() >= 4),
            df_asignacion_test['AGENCIA_CORRECTA'],
            df_asignacion_test['AGENCIA']
        )
        df_asignacion_test.drop(columns=['AGENCIA_CORRECTA'], inplace=True)

cols_asignacion = ['CODIGO', 'CONT_18', 'NOMBRE', 'TIPO_CARTERA', 'TIPO_FONDO', 'CLAVE', 'AGENCIA']
df_asignacion_test = df_asignacion_test[cols_asignacion]

df_asignacion_test['CONT_18'] = df_asignacion_test['CONT_18'].apply(lambda x: str(int(x)).zfill(18) if pd.notna(x) else x)
df_asignacion_test['CODIGO'] = df_asignacion_test['CODIGO'].astype('Int64').astype(str).str.zfill(8)

conteo_cc = df_asignacion_test['CODIGO'].value_counts()
df_asignacion_test['FLAG'] = df_asignacion_test['CODIGO'].map(conteo_cc)

print(df_asignacion_test.shape)
df_asignacion_test.head()

In [ ]:
df_asignacion_test['AGENCIA'].value_counts()

In [6]:
df_asignacion_test.to_excel(asignacion_final_path, index=False)